# Facebook Down on October 2021

### Overview
Facebook was down on 4th of October 2021 for 6 hours (15:40 UTC to 21:40 UTC)

This notebook is a support for https://ganymede.cloud/facebook-down.html blog post

## Related Blogpost

- Facebook blog post: https://engineering.fb.com/2021/10/04/networking-traffic/outage/
- Global context: https://www.nytimes.com/2021/10/04/technology/facebook-down.html
- Technical context: https://blog.cloudflare.com/october-2021-facebook-outage/



# Get Facebook trades for 4th of October

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_pb2 as tick_trades
import systemathics.apis.services.tick.v1.tick_trades_pb2_grpc as tick_trades_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data
To request *tick trades* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'FB' # Facebook
exchange = 'XNGS' # Nasdaq

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 10, day = 4), 
    end_date = date.Date(year = 2021, month = 10, day = 4)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

In [ ]:
# generate the tick trades request
identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades.TickTradesRequest(
    identifiers = [identifier],
    constraints = constraints
)

In [ ]:
# open a gRPC channel
with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
    credentials = grpc.ssl_channel_credentials(f.read())
with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:
    
    # instantiate the tick trades service
    service = tick_trades_service.TickTradesServiceStub(channel)
    
    # process the tick trades request
    trades = []
    metadata = [('authorization', token)]
    for trade in service.TickTrades(request=request, metadata=metadata):
        if trade.trade:
            trades.append(trade.trade)

# keep only trades, delete instrument mapping (1st element retrieved)
# mapping is not used as only one instrument identifier is requested
del trades[0]

The following code snippet displays the total retrieved trades count:

In [ ]:
print('Total trades retrieved: {0}'.format(len(trades)))

### Step 4: Dataframe creation
Let's create 3 dataframe.
- One before the outage (15:40 UTC)
- During outage (between 15:40 and 21:40 UTC)
- after outage (after 21:40 UTC)

In [ ]:
# prepare the dataframe content
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in trades]
prices = [t.price for t in trades]
sizes = [t.size for t in trades]

In [ ]:
# create a pandas dataframe with: dates, trades prices and sizes
d = {'Date': dates, 'Price': prices, 'Size': sizes}
df = pd.DataFrame(data=d)
df

In [ ]:
outage = df[ (df['Date'] > datetime(2021,10,4,15,40,0)) & (df['Date'] < datetime(2021,10,4,21,40,0)) ]
before_outage = df[ (df['Date'] <= datetime(2021,10,4,15,40,0)) ]
after_outage = df[(df['Date'] >= datetime(2021,10,4,21,40,0)) ]


### Step 5: Visualize data

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'Price', data=before_outage, marker='', color='blue', linewidth=2, label="Price")
ax.plot('Date', 'Price', data=outage, marker='', color='red', linewidth=2, label="Price")
ax.plot('Date', 'Price', data=after_outage, marker='', color='blue', linewidth=2, label="Price")
# twin x-axis for two different y-axis
ax2=ax.twinx()
ax2.plot('Date', 'Size', data=df, marker='', color='green', linewidth=2, label="Size")

# set graph title and axis label
ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="blue",fontsize=14)
ax2.set_ylabel("Size",color="green",fontsize=14)
plt.title('Tick trades for {0}-{1}'.format(ticker, exchange))
plt.show()

## Conclusion

- Facebook price before outage decreased significantly.
- Price during the outage barely changed.
- We can conclude that Facebook outage did not affect it's stock price. Initial decrease was likely due to whistle-blower Frances Haugen as explained by NY Times article.